In [24]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [9]:
#Load Yelp API Credentials
with open('/Users/Yeajoon Yoo/codingdojo/.secret/yelp_api.json') as f: #my pathway
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [11]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [13]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
type(results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [14]:
LOCATION = 'Lynchburg'
TERM = 'Steak'

In [15]:
JSON_FILE = f"Data/results_Lynchburg_Steak.json"
JSON_FILE

'Data/results_Lynchburg_Steak.json'

In [16]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    
    if len(folder)>0:
        os.makedirs(folder,exist_ok=True)
        
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_Lynchburg_Steak.json not found. Saving empty list to file.


In [17]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [18]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [19]:
total_results = results['total']
total_results

161

In [20]:
results_per_page = len(results['businesses'])
results_per_page

20

In [21]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

9

In [22]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [25]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [26]:
# add progress bar
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/9 [00:00<?, ?it/s]

In [27]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/9 [00:00<?, ?it/s]

In [28]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ThUfjlii1hY3Yyx2GY7D9A,iron-and-ale-lynchburg-tap-and-table-lynchburg,Iron & Ale Lynchburg Tap and Table,https://s3-media2.fl.yelpcdn.com/bphoto/Hp7ez9...,False,https://www.yelp.com/biz/iron-and-ale-lynchbur...,227,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 37.33946, 'longitude': -79.2248}","[pickup, delivery]",$$,"{'address1': '106 Cornerstone St', 'address2':...",+14349449697,(434) 944-9697,5891.180237
1,9kiHCpIDcdyjK9PkxnaFLQ,bulls-steakhouse-forest,Bulls Steakhouse,https://s3-media2.fl.yelpcdn.com/bphoto/c9VhKC...,False,https://www.yelp.com/biz/bulls-steakhouse-fore...,48,"[{'alias': 'steak', 'title': 'Steakhouses'}]",4.0,"{'latitude': 37.37881, 'longitude': -79.24887}",[delivery],$$,"{'address1': '1887 Graves Mill Rd', 'address2'...",+14343857581,(434) 385-7581,4910.241809
2,P_ohsmG-EUMBregPCcyphw,william-and-henry-steakhouse-lynchburg,William & Henry Steakhouse,https://s3-media1.fl.yelpcdn.com/bphoto/Cch7OF...,False,https://www.yelp.com/biz/william-and-henry-ste...,55,"[{'alias': 'steak', 'title': 'Steakhouses'}]",4.0,"{'latitude': 37.41535916319549, 'longitude': -...",[delivery],NaN,"{'address1': '712 Church St', 'address2': None...",+14346164423,(434) 616-4423,5317.230855
3,cJR6sta1Md3QoTq3xhQq0w,shoemakers-restaurant-lynchburg,Shoemaker's Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/YHefR9...,False,https://www.yelp.com/biz/shoemakers-restaurant...,213,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 37.411627, 'longitude': -79.137913}",[delivery],$$$,"{'address1': '1312 Commerce St', 'address2': '...",+14344551510,(434) 455-1510,5509.902062
4,MLbP36BXbNefVgVxGjJp9w,benjamins-forest,Benjamin's,https://s3-media1.fl.yelpcdn.com/bphoto/0GKIJm...,False,https://www.yelp.com/biz/benjamins-forest?adju...,100,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 37.36628, 'longitude': -79.29494}",[delivery],$$,"{'address1': '14900 Forest Rd', 'address2': ''...",+14345346077,(434) 534-6077,9440.648026


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
156,J6_Wsg3XLdF-fK2MWYUVVg,subway-rustburg-4,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/PXUkgo...,False,https://www.yelp.com/biz/subway-rustburg-4?adj...,1,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",1.0,"{'latitude': 37.2793513, 'longitude': -79.1681...",[],$,"{'address1': '1788 Calohan Rd', 'address2': ''...",+14346162266,(434) 616-2266,11960.781906
157,c9QYtmzkh8usQG2TsrKrOg,la-caretta-forest,La Caretta,https://s3-media2.fl.yelpcdn.com/bphoto/cvhD6e...,False,https://www.yelp.com/biz/la-caretta-forest?adj...,30,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 37.3694915771484, 'longitude': -7...",[delivery],$,"{'address1': '14575 Forest Rd', 'address2': ''...",+14345254737,(434) 525-4737,9902.484854
158,xu_cfWtJcq9UWKudEquWoA,subway-forest-7,Subway,https://s3-media3.fl.yelpcdn.com/bphoto/-5ffD4...,False,https://www.yelp.com/biz/subway-forest-7?adjus...,5,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",1.5,"{'latitude': 37.3648674699833, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '14805 Forest Rd', 'address2': 'S...",+14345253240,(434) 525-3240,9547.443217
159,iwNaYH6YEZs28iCQm8_fhQ,subway-madison-heights-5,Subway,https://s3-media1.fl.yelpcdn.com/bphoto/deFB42...,False,https://www.yelp.com/biz/subway-madison-height...,11,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",2.5,"{'latitude': 37.4492634712345, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '4574 S Amherst Hwy', 'address2':...",+14348457827,(434) 845-7827,9414.563201
160,aEHbJKaYZ-Mves0f4Tjujg,dominos-pizza-forest-4,Domino's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/LRm23C...,False,https://www.yelp.com/biz/dominos-pizza-forest-...,14,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",1.5,"{'latitude': 37.3700233, 'longitude': -79.3017...",[],$,"{'address1': '14521 Forest Rd', 'address2': ''...",+14343850404,(434) 385-0404,9949.106678


In [29]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

1

In [30]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_crab_cakes.csv.gz', compression='gzip',index=False)